In [125]:
import pandas as pd

In [126]:
def prepare(data):
    data['import.start'] = pd.to_datetime(data['import.start'])
    data['import.stop'] = pd.to_datetime(data['import.stop'])
    data['created'] = pd.to_datetime(data['created'])
    data['started'] = pd.to_datetime(data['started'])
    data['finished'] = pd.to_datetime(data['finished'])
    
    data['starttime'] = (data['started'] - data['created']).dt.total_seconds() 
    data['dockertime'] = (data['finished'] - data['started']).dt.total_seconds() 
    data['importtime'] = (data['import.stop'] - data['import.start']).dt.total_seconds() 
    
    started_temp = pd.DataFrame()
    docker_temp = pd.DataFrame()
    import_temp = pd.DataFrame()
    
    for i in range(8,38,6):
        started_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+1]) - pd.to_datetime(data.iloc[:, i])).dt.total_seconds() , True)
        docker_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+2]) - pd.to_datetime(data.iloc[:, i+1])).dt.total_seconds() , True)
        import_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+4]) - pd.to_datetime(data.iloc[:, i+3])).dt.total_seconds() , True)
        
    
    started_temp.columns = ['starttime{}'.format(i) for i in range(1, 6)]
    docker_temp.columns = ['dockertime{}'.format(i) for i in range(1, 6)]
    import_temp.columns = ['importtime{}'.format(i) for i in range(1, 6)]
    
    data = pd.concat([data, started_temp, docker_temp, import_temp], axis=1)
    
    return data

In [127]:
def extract_imports(imp):
    imp = imp.split("'")
    l = ""
    
    for i in imp:
        if i == "(" or i == ")" or i == ",)" or i == "," or i == "()" or i == ", ":
            continue
        else:
            if l == "":
                l = i
            else:
                l = l + ";" + i
    return l

In [165]:
def repair(data, install_libaries, approach):
    for i in range(0,207):
        if install_libaries[approach][i] == 0 :
            if data.installtime.iloc[i] > -1 :
                data.buldingtime.iloc[i] = data.buldingtime.iloc[i] - data.installtime.iloc[i]
                data.installtime.iloc[i] = -1

This is one experiment run with using the top 10 packages over all functions
The installationtime is set to minus -1 if there was no install as for native python packages as os

In [180]:
approach = "all_top_10"
#approach = "vanilla_approach"
#approach = "cluster_top_10"
col = ['imports', 'buldingtime', 'installtime', 'starttime', 'dockertime',
       'importtime', 'starttime1', 'starttime2', 'starttime3', 'starttime4',
       'starttime5', 'dockertime1', 'dockertime2', 'dockertime3',
       'dockertime4', 'dockertime5', 'importtime1', 'importtime2',
       'importtime3', 'importtime4', 'importtime5']

In [152]:
install = pd.read_csv("experiments/installations.csv")
install_libaries = pd.DataFrame()
for index, row in install.iterrows():
    if pd.isna(row[1]):
        ins = set()
    else: 
        ins = set(row[1].split(";"))
    all = set(row[2].split(";"))
    if pd.isna(row[3]):
        clu = set()
    else: 
        clu = set(row[3].split(";"))
    install_libaries = pd.concat([install_libaries, pd.DataFrame([[row[0], len(ins), len(ins)-0, len(ins)-len(ins.intersection(all)), len(ins)-len(ins.intersection(clu))]])], ignore_index=True)
install_libaries.columns = ['imports', 'toInstall', 'vanilla_approach', 'all_top_10', 'cluster_top_10']

install_libaries

/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_8026/4029374880.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(row[1]):
/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_8026/4029374880.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ins = set(row[1].split(";"))
/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_8026/4029374880.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  all = 

,imports,toInstall,vanilla_approach,all_top_10,cluster_top_10
0,argparse;copy;datetime;funcx;itertools;json;ma...,7,7,4,0
1,argparse;copy;funcx;itertools;json;matplotlib;...,6,6,4,0
2,argparse;sys,1,1,1,0
3,CLEAN,1,1,1,0
4,CLEAN;os,1,1,1,0
...,...,...,...,...,...
202,timeit,0,0,0,0
203,torch,1,1,0,0
204,torch;torchvision,2,2,1,2
205,traceback,0,0,0,0


In [181]:
data = pd.read_csv("data/"+approach+"/run10.csv")
data = prepare(data)
imp = data[col[0]]
data = data[col[1:]]

In [182]:
for i in range(9,0,-1):
    tmp = pd.read_csv("data/"+approach+"/run"+str(i)+".csv")
    tmp = prepare(tmp)
    tmp = tmp[col[1:]]
    data = (data + tmp)
data = data/10
data.insert(0, col[0],imp)

In [183]:
data

,imports,buldingtime,installtime,starttime,dockertime,importtime,starttime1,starttime2,starttime3,starttime4,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
0,argparse,8.41,5.31,2.026819,1.210094,0.044723,0.643705,0.622289,0.749516,0.647798,...,1.033787,1.033656,1.033979,1.033972,1.033666,0.010771,0.011305,0.010256,0.010476,0.009689
1,clean,10.11,6.21,2.264412,1.225619,0.008079,0.806776,0.811559,0.876272,0.895461,...,1.020749,1.020595,1.020425,1.020686,1.020643,0.000363,0.000313,0.000283,0.000356,0.000386
2,collections,3.10,-1.00,2.838753,1.332040,0.053543,0.755990,0.707457,0.751481,0.635921,...,1.055140,1.055973,1.055858,1.055842,1.055403,0.017491,0.017785,0.017393,0.017150,0.017925
3,colmena,18.33,14.83,2.292351,1.201289,0.008861,0.774815,0.751169,0.748512,0.698446,...,1.020600,1.020932,1.020517,1.020675,1.020727,0.000348,0.000371,0.000343,0.000450,0.000418
4,common,10.46,7.22,2.216309,1.175314,0.007103,0.655773,0.733643,0.763878,0.684046,...,1.021186,1.020317,1.021178,1.020223,1.020748,0.000388,0.000415,0.000325,0.000255,0.000361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,urllib3,8.46,5.57,2.105103,1.662203,0.455932,0.718015,0.659613,0.754252,0.807083,...,1.113635,1.112219,1.112764,1.113489,1.113261,0.079703,0.079253,0.079546,0.079558,0.079631
88,uuid,9.98,7.12,2.033960,1.218466,0.052394,0.635997,0.656979,0.617127,0.668849,...,1.034703,1.034570,1.034552,1.034244,1.034420,0.011224,0.011740,0.011830,0.012209,0.011898
89,warnings,2.65,-1.00,2.180663,1.233780,0.000008,0.768780,0.746880,0.726569,0.775309,...,1.036572,1.035644,1.036465,1.037282,1.035259,0.000010,0.000012,0.000008,0.000008,0.000007
90,webbrowser,2.68,-1.00,2.186071,1.513124,0.239609,0.743873,0.669989,0.748356,0.741852,...,1.160715,1.167929,1.168152,1.164633,1.162913,0.119145,0.124510,0.119990,0.123114,0.121590


In [175]:
repair(data, install_libaries, approach)

/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_8026/795380242.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data.buldingtime.iloc[i] = data.buldingtime.iloc[i] - data.installtime.iloc[i]
/var/folders/86/fyd29lqj0033vg6drb2b

This is the mapping between imports and function
For this merge and the following ones, there is no need to think about on which join-operation. I cleaned the data beforehand. That is, there will be always a pair 
```python
pd.merge(func_mapping,data, on='imports')
```

In [132]:
func_mapping = pd.read_csv('experiments/function_mapping.csv')
func_mapping = func_mapping.drop(2)
func_mapping['func4.imports'] = func_mapping[func_mapping['func4.imports'].str.len() >= 0]['func4.imports'].map(lambda x: extract_imports(x))
func_mapping = func_mapping[func_mapping['func4.imports'] != ""]
func_mapping.rename(columns={'func4.imports': 'imports'}, inplace=True)
func_mapping.drop(columns=['Unnamed: 0'], inplace=True)

In [133]:
func_mapping.head()

,Unnamed: 0.1,func4.name,imports,Cluster
0,0,0-04-_check_slurm_job_statuses.py,subprocess,Job
3,3,873-04-call_mtrack.py,mtrack;sys,Data:_mtrack
4,4,43-04-make_corr_plots.py,json;os;xpcs_webplot,Sci:_Instruments
6,6,3-04-submit_slurm_job.py,os;subprocess,Job
7,7,397-01-rejection_sample.py,random,Trivial


Here is the mapping between functions and task
```python
pd.merge(func_mapping,task_mapping, on='func4.name')
```

In [134]:
task_mapping = pd.read_csv("funcx/task_function_mapping.csv")

In [135]:
task_mapping.head()

,task_uuid,func4.name,executiontime
0,000001d5-b3f0-4c17-8074-04f013f7bf52,0-04-_check_slurm_job_statuses.py,0.0243
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,0-04-_check_slurm_job_statuses.py,0.0331
2,00002642-f850-42b4-9e1e-2698c3396498,0-04-_check_slurm_job_statuses.py,0.0432
3,00002deb-cf79-43ed-8aff-3281938809fe,0-04-_check_slurm_job_statuses.py,0.0268
4,0000754b-898e-43dc-9b55-35d1a6fe6633,0-04-_check_slurm_job_statuses.py,0.0284


Here we have the timestamp for each task
For merging
```python
pd.merge(task_mapping,task_mapping, on='task_uuid')
```

In [136]:
task = pd.read_csv("funcx/tasks_pt1.csv")
task = pd.concat([task, pd.read_csv("funcx/tasks_pt2.csv")], ignore_index=True)
task['received'] = task['received'].interpolate()

In [137]:
task.head()

,task_uuid,user_id,endpoint_uuid,received
0,000001d5-b3f0-4c17-8074-04f013f7bf52,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.681417e+18
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679386e+18
2,00002642-f850-42b4-9e1e-2698c3396498,286.0,cbc1f8d4-71b2-4a41-8ed4-9c75421eb89f,1.683246e+18
3,00002deb-cf79-43ed-8aff-3281938809fe,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.677897e+18
4,0000754b-898e-43dc-9b55-35d1a6fe6633,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679277e+18


Here are information about the endpoints
For merging:
```python
pd.merge(task,endpoints, on='endpoint_uuid')
```

In [138]:
endpoints = pd.read_csv("funcx/endpoints.csv")

In [139]:
endpoints.head()

,endpoint_uuid,X_type,ip,cluster
0,00000000-0000-4000-8000-000011110000,LocalProvider,216.80.70.252,6
1,001fecc2-1393-4bd9-950e-5645e15e3a53,SlurmProvider,141.142.140.194,10
2,006f558d-df82-45c2-b2d5-94274ef41a69,SlurmProvider,141.142.140.194,10
3,00824163-569c-4f0d-953d-097cd3b7179d,NaN,159.226.43.47,10
4,01a3324e-7ad9-4563-b598-0c2499068a39,LocalProvider,128.135.204.6,8


Example merging in the end:
```python
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
```

In [140]:
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
r.to_csv("my_data.csv")

In [141]:
sorted_df = r.sort_values(by=["received"], ascending=True)
sorted_df = sorted_df.dropna()
sorted_df.to_csv(approach+"_data_sorted.csv")

In [142]:
sorted_df.head()

,task_uuid,user_id,endpoint_uuid,received,func4.name,executiontime,Unnamed: 0.1,imports,Cluster,buldingtime,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
1109608,2dda391f-cb85-4e2f-8ae7-c9c603a821dd,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669597e+18,296-04-tar.py,0.0021,436,os;pathlib;tarfile,Sci:_Instruments,8.22,...,1.047908,1.047566,1.047872,1.048078,1.047729,0.015908,0.01572,0.016456,0.016295,0.01513
1109630,ac3f3e5e-aeff-4236-a3a5-223dcf00aba4,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669598e+18,296-04-tar.py,0.0017,436,os;pathlib;tarfile,Sci:_Instruments,8.22,...,1.047908,1.047566,1.047872,1.048078,1.047729,0.015908,0.01572,0.016456,0.016295,0.01513
1109604,1347620d-56a1-4d30-b8d3-32ded9e08b47,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669598e+18,296-04-tar.py,0.0020,436,os;pathlib;tarfile,Sci:_Instruments,8.22,...,1.047908,1.047566,1.047872,1.048078,1.047729,0.015908,0.01572,0.016456,0.016295,0.01513
1109605,1fbeacd4-b3e9-4bb9-bc1c-92fc9abd238f,600.0,674446c2-4dc6-41a6-b767-bf6ce67df702,1.669598e+18,296-04-tar.py,0.0095,436,os;pathlib;tarfile,Sci:_Instruments,8.22,...,1.047908,1.047566,1.047872,1.048078,1.047729,0.015908,0.01572,0.016456,0.016295,0.01513
1109606,285a505d-66ae-4e5c-88a3-e70794e425ae,600.0,674446c2-4dc6-41a6-b767-bf6ce67df702,1.669598e+18,296-04-tar.py,0.0081,436,os;pathlib;tarfile,Sci:_Instruments,8.22,...,1.047908,1.047566,1.047872,1.048078,1.047729,0.015908,0.01572,0.016456,0.016295,0.01513


In [143]:
p = sorted_df.dropna()

In [144]:
p[p.task_uuid.isna()]

,task_uuid,user_id,endpoint_uuid,received,func4.name,executiontime,Unnamed: 0.1,imports,Cluster,buldingtime,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
